In [3]:
import pandas as pd
df = pd.read_csv("D:\G95-Dataset\phase2cleaneddataset.csv",encoding='cp1252')
start_Stime = 1421927443       
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 600))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.add_node(src_node)
    attackG_10_4_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.add_node(src_node)
    nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_10_4_10_3_10_2_10_1_10_2_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_10_4_10_3_10_2_10_1_10_2_10_1.add_node(src_node)
    overallG_10_4_10_3_10_2_10_1_10_2_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_10_4_10_3_10_2_10_1_10_2_10_1.add_edge(src_node, dst_node, weight=edge_weight)


C:\Users\HP\AppData\Local\Temp\ipykernel_12512\2160805178.py:2: DtypeWarning: Columns (1,3,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:\G95-Dataset\phase2cleaneddataset.csv",encoding='cp1252')


In [4]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_10_4_10_3_10_2_10_1_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_10_4_10_3_10_2_10_1_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_clustering_non_attack = nx.average_clustering(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_clustering_overall = nx.average_clustering(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.degree()).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1_10_2_10_1.degree()).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_10_4_10_3_10_2_10_1_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_10_4_10_3_10_2_10_1_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.in_degree()).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_in_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.in_degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_in_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1_10_2_10_1.in_degree()).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.out_degree()).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_out_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1.out_degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_out_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1_10_2_10_1.out_degree()).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_pagerank_non_attack = sum(nx.pagerank(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_pagerank_overall = sum(nx.pagerank(overallG_10_4_10_3_10_2_10_1_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_shortest_path_non_attack = calculate_diameter(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
avg_shortest_path_overall = calculate_diameter(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
density_non_attack = nx.density(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
density_overall = nx.density(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
diameter_non_attack = calculate_diameter(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
diameter_overall = calculate_diameter(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_louvain_non_attack = detect_communities(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_louvain_overall = detect_communities(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(attackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
reciprocity_non_attack = calculate_reciprocity(nonattackG_10_4_10_3_10_2_10_1_10_1_10_2_10_1_10_1)
reciprocity_overall = calculate_reciprocity(overallG_10_4_10_3_10_2_10_1_10_2_10_1)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.07492264242550674
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.054382308843330424
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 3.122657616455456e-11
Average closeness centrality for attack graph: 0.008767189095057935
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0003419674400073852
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.191256830601093
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.7361319340329837
Average degree centrality for attack graph: 0.017534378190115878
Average degree centrality for non-attack graph: 0.0
Average degree c

In [13]:
start_Stime = 1421927444    
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 600))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
attackG_10_4_10_3_10_2_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG_10_4_10_3_10_2_10_1.add_node(src_node)
    attackG_10_4_10_3_10_2_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG_10_4_10_3_10_2_10_1.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonattackG_10_4_10_3_10_2_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonattackG_10_4_10_3_10_2_10_1.add_node(src_node)
    nonattackG_10_4_10_3_10_2_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_10_4_10_3_10_2_10_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_10_4_10_3_10_2_10_1.add_node(src_node)
    overallG_10_4_10_3_10_2_10_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_10_4_10_3_10_2_10_1.add_edge(src_node, dst_node, weight=edge_weight)

In [14]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(attackG_10_4_10_3_10_2_10_1)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonattackG_10_4_10_3_10_2_10_1)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_10_4_10_3_10_2_10_1)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(attackG_10_4_10_3_10_2_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonattackG_10_4_10_3_10_2_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_10_4_10_3_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(attackG_10_4_10_3_10_2_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonattackG_10_4_10_3_10_2_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_10_4_10_3_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(attackG_10_4_10_3_10_2_10_1)
avg_clustering_non_attack = nx.average_clustering(nonattackG_10_4_10_3_10_2_10_1)
avg_clustering_overall = nx.average_clustering(overallG_10_4_10_3_10_2_10_1)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1.degree()).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1.degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1.degree()).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(attackG_10_4_10_3_10_2_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonattackG_10_4_10_3_10_2_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_10_4_10_3_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(attackG_10_4_10_3_10_2_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonattackG_10_4_10_3_10_2_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_10_4_10_3_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1.in_degree()).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_in_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1.in_degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_in_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1.in_degree()).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(attackG_10_4_10_3_10_2_10_1.out_degree()).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_out_degree_non_attack = sum(dict(nonattackG_10_4_10_3_10_2_10_1.out_degree()).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_out_degree_overall = sum(dict(overallG_10_4_10_3_10_2_10_1.out_degree()).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(attackG_10_4_10_3_10_2_10_1).values()) / len(attackG_10_4_10_3_10_2_10_1)
avg_pagerank_non_attack = sum(nx.pagerank(nonattackG_10_4_10_3_10_2_10_1).values()) / len(nonattackG_10_4_10_3_10_2_10_1)
avg_pagerank_overall = sum(nx.pagerank(overallG_10_4_10_3_10_2_10_1).values()) / len(overallG_10_4_10_3_10_2_10_1)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(attackG_10_4_10_3_10_2_10_1)
avg_shortest_path_non_attack = calculate_diameter(nonattackG_10_4_10_3_10_2_10_1)
avg_shortest_path_overall = calculate_diameter(overallG_10_4_10_3_10_2_10_1)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(attackG_10_4_10_3_10_2_10_1)
density_non_attack = nx.density(nonattackG_10_4_10_3_10_2_10_1)
density_overall = nx.density(overallG_10_4_10_3_10_2_10_1)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(attackG_10_4_10_3_10_2_10_1)
diameter_non_attack = calculate_diameter(nonattackG_10_4_10_3_10_2_10_1)
diameter_overall = calculate_diameter(overallG_10_4_10_3_10_2_10_1)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(attackG_10_4_10_3_10_2_10_1)
num_louvain_non_attack = detect_communities(nonattackG_10_4_10_3_10_2_10_1)
num_louvain_overall = detect_communities(overallG_10_4_10_3_10_2_10_1)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(attackG_10_4_10_3_10_2_10_1)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonattackG_10_4_10_3_10_2_10_1)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_10_4_10_3_10_2_10_1)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(attackG_10_4_10_3_10_2_10_1)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonattackG_10_4_10_3_10_2_10_1)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_10_4_10_3_10_2_10_1)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(attackG_10_4_10_3_10_2_10_1)
reciprocity_non_attack = calculate_reciprocity(nonattackG_10_4_10_3_10_2_10_1)
reciprocity_overall = calculate_reciprocity(overallG_10_4_10_3_10_2_10_1)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.07492264242550674
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.05402276678235315
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 3.1086511984455566e-11
Average closeness centrality for attack graph: 0.008767189095057935
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0003408196916208675
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.191256830601093
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.7310379241516967
Average degree centrality for attack graph: 0.017534378190115878
Average degree centrality for non-attack graph: 0.0
Average degree c

In [23]:
start_Stime = 1421927644       
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 600))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
attackG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG_10_4_10_3.add_node(src_node)
    attackG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG_10_4_10_3.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonattackG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonattackG_10_4_10_3.add_node(src_node)
    nonattackG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_10_4_10_3.add_node(src_node)
    overallG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_10_4_10_3.add_edge(src_node, dst_node, weight=edge_weight)

In [24]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(attackG_10_4_10_3)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonattackG_10_4_10_3)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_10_4_10_3)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(attackG_10_4_10_3)
avg_clustering_non_attack = nx.average_clustering(nonattackG_10_4_10_3)
avg_clustering_overall = nx.average_clustering(overallG_10_4_10_3)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(attackG_10_4_10_3.degree()).values()) / len(attackG_10_4_10_3)
avg_degree_non_attack = sum(dict(nonattackG_10_4_10_3.degree()).values()) / len(nonattackG_10_4_10_3)
avg_degree_overall = sum(dict(overallG_10_4_10_3.degree()).values()) / len(overallG_10_4_10_3)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(attackG_10_4_10_3.in_degree()).values()) / len(attackG_10_4_10_3)
avg_in_degree_non_attack = sum(dict(nonattackG_10_4_10_3.in_degree()).values()) / len(nonattackG_10_4_10_3)
avg_in_degree_overall = sum(dict(overallG_10_4_10_3.in_degree()).values()) / len(overallG_10_4_10_3)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(attackG_10_4_10_3.out_degree()).values()) / len(attackG_10_4_10_3)
avg_out_degree_non_attack = sum(dict(nonattackG_10_4_10_3.out_degree()).values()) / len(nonattackG_10_4_10_3)
avg_out_degree_overall = sum(dict(overallG_10_4_10_3.out_degree()).values()) / len(overallG_10_4_10_3)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_pagerank_non_attack = sum(nx.pagerank(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_pagerank_overall = sum(nx.pagerank(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(attackG_10_4_10_3)
avg_shortest_path_non_attack = calculate_diameter(nonattackG_10_4_10_3)
avg_shortest_path_overall = calculate_diameter(overallG_10_4_10_3)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(attackG_10_4_10_3)
density_non_attack = nx.density(nonattackG_10_4_10_3)
density_overall = nx.density(overallG_10_4_10_3)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(attackG_10_4_10_3)
diameter_non_attack = calculate_diameter(nonattackG_10_4_10_3)
diameter_overall = calculate_diameter(overallG_10_4_10_3)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(attackG_10_4_10_3)
num_louvain_non_attack = detect_communities(nonattackG_10_4_10_3)
num_louvain_overall = detect_communities(overallG_10_4_10_3)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(attackG_10_4_10_3)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonattackG_10_4_10_3)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_10_4_10_3)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(attackG_10_4_10_3)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonattackG_10_4_10_3)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_10_4_10_3)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(attackG_10_4_10_3)
reciprocity_non_attack = calculate_reciprocity(nonattackG_10_4_10_3)
reciprocity_overall = calculate_reciprocity(overallG_10_4_10_3)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.034921686098243414
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.029736207709790963
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 2.6492256906732717e-09
Average closeness centrality for attack graph: 0.008334281488223929
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0002481602099167524
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.117021276595745
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.5633423180592994
Average degree centrality for attack graph: 0.016668562976447854
Average degree centrality for non-attack graph: 0.0
Average degree

In [29]:
start_Stime = 1424231676      
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 600))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
attackG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG_10_4_10_3.add_node(src_node)
    attackG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG_10_4_10_3.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonattackG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonattackG_10_4_10_3.add_node(src_node)
    nonattackG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_10_4_10_3 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_10_4_10_3.add_node(src_node)
    overallG_10_4_10_3.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_10_4_10_3.add_edge(src_node, dst_node, weight=edge_weight)

In [30]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(attackG_10_4_10_3)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonattackG_10_4_10_3)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_10_4_10_3)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(attackG_10_4_10_3)
avg_clustering_non_attack = nx.average_clustering(nonattackG_10_4_10_3)
avg_clustering_overall = nx.average_clustering(overallG_10_4_10_3)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(attackG_10_4_10_3.degree()).values()) / len(attackG_10_4_10_3)
avg_degree_non_attack = sum(dict(nonattackG_10_4_10_3.degree()).values()) / len(nonattackG_10_4_10_3)
avg_degree_overall = sum(dict(overallG_10_4_10_3.degree()).values()) / len(overallG_10_4_10_3)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(attackG_10_4_10_3.in_degree()).values()) / len(attackG_10_4_10_3)
avg_in_degree_non_attack = sum(dict(nonattackG_10_4_10_3.in_degree()).values()) / len(nonattackG_10_4_10_3)
avg_in_degree_overall = sum(dict(overallG_10_4_10_3.in_degree()).values()) / len(overallG_10_4_10_3)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(attackG_10_4_10_3.out_degree()).values()) / len(attackG_10_4_10_3)
avg_out_degree_non_attack = sum(dict(nonattackG_10_4_10_3.out_degree()).values()) / len(nonattackG_10_4_10_3)
avg_out_degree_overall = sum(dict(overallG_10_4_10_3.out_degree()).values()) / len(overallG_10_4_10_3)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(attackG_10_4_10_3).values()) / len(attackG_10_4_10_3)
avg_pagerank_non_attack = sum(nx.pagerank(nonattackG_10_4_10_3).values()) / len(nonattackG_10_4_10_3)
avg_pagerank_overall = sum(nx.pagerank(overallG_10_4_10_3).values()) / len(overallG_10_4_10_3)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(attackG_10_4_10_3)
avg_shortest_path_non_attack = calculate_diameter(nonattackG_10_4_10_3)
avg_shortest_path_overall = calculate_diameter(overallG_10_4_10_3)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(attackG_10_4_10_3)
density_non_attack = nx.density(nonattackG_10_4_10_3)
density_overall = nx.density(overallG_10_4_10_3)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(attackG_10_4_10_3)
diameter_non_attack = calculate_diameter(nonattackG_10_4_10_3)
diameter_overall = calculate_diameter(overallG_10_4_10_3)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(attackG_10_4_10_3)
num_louvain_non_attack = detect_communities(nonattackG_10_4_10_3)
num_louvain_overall = detect_communities(overallG_10_4_10_3)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(attackG_10_4_10_3)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonattackG_10_4_10_3)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_10_4_10_3)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(attackG_10_4_10_3)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonattackG_10_4_10_3)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_10_4_10_3)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(attackG_10_4_10_3)
reciprocity_non_attack = calculate_reciprocity(nonattackG_10_4_10_3)
reciprocity_overall = calculate_reciprocity(overallG_10_4_10_3)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.0768494260250417
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.036848056757459564
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 5.308060147236809e-11
Average closeness centrality for attack graph: 0.008624923265807228
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0002836071128754182
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.1049723756906076
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.5800922874093604
Average degree centrality for attack graph: 0.017249846531614467
Average degree centrality for non-attack graph: 0.0
Average degree c

In [27]:
start_Stime = 1424231076   
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 600))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
attackG_10_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    attackG_10_4.add_node(src_node)
    attackG_10_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    attackG_10_4.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonattackG_10_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonattackG_10_4.add_node(src_node)
    nonattackG_10_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_10_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_10_4.add_node(src_node)
    overallG_10_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_10_4.add_edge(src_node, dst_node, weight=edge_weight)

In [28]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(attackG_10_4)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonattackG_10_4)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_10_4)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(attackG_10_4).values()) / len(attackG_10_4)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonattackG_10_4).values()) / len(nonattackG_10_4)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_10_4).values()) / len(overallG_10_4)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(attackG_10_4).values()) / len(attackG_10_4)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonattackG_10_4).values()) / len(nonattackG_10_4)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_10_4).values()) / len(overallG_10_4)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(attackG_10_4)
avg_clustering_non_attack = nx.average_clustering(nonattackG_10_4)
avg_clustering_overall = nx.average_clustering(overallG_10_4)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(attackG_10_4.degree()).values()) / len(attackG_10_4)
avg_degree_non_attack = sum(dict(nonattackG_10_4.degree()).values()) / len(nonattackG_10_4)
avg_degree_overall = sum(dict(overallG_10_4.degree()).values()) / len(overallG_10_4)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(attackG_10_4).values()) / len(attackG_10_4)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonattackG_10_4).values()) / len(nonattackG_10_4)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_10_4).values()) / len(overallG_10_4)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(attackG_10_4).values()) / len(attackG_10_4)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonattackG_10_4).values()) / len(nonattackG_10_4)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_10_4).values()) / len(overallG_10_4)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(attackG_10_4.in_degree()).values()) / len(attackG_10_4)
avg_in_degree_non_attack = sum(dict(nonattackG_10_4.in_degree()).values()) / len(nonattackG_10_4)
avg_in_degree_overall = sum(dict(overallG_10_4.in_degree()).values()) / len(overallG_10_4)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(attackG_10_4.out_degree()).values()) / len(attackG_10_4)
avg_out_degree_non_attack = sum(dict(nonattackG_10_4.out_degree()).values()) / len(nonattackG_10_4)
avg_out_degree_overall = sum(dict(overallG_10_4.out_degree()).values()) / len(overallG_10_4)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(attackG_10_4).values()) / len(attackG_10_4)
avg_pagerank_non_attack = sum(nx.pagerank(nonattackG_10_4).values()) / len(nonattackG_10_4)
avg_pagerank_overall = sum(nx.pagerank(overallG_10_4).values()) / len(overallG_10_4)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(attackG_10_4)
avg_shortest_path_non_attack = calculate_diameter(nonattackG_10_4)
avg_shortest_path_overall = calculate_diameter(overallG_10_4)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(attackG_10_4)
density_non_attack = nx.density(nonattackG_10_4)
density_overall = nx.density(overallG_10_4)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(attackG_10_4)
diameter_non_attack = calculate_diameter(nonattackG_10_4)
diameter_overall = calculate_diameter(overallG_10_4)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(attackG_10_4)
num_louvain_non_attack = detect_communities(nonattackG_10_4)
num_louvain_overall = detect_communities(overallG_10_4)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(attackG_10_4)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonattackG_10_4)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_10_4)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(attackG_10_4)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonattackG_10_4)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_10_4)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(attackG_10_4)
reciprocity_non_attack = calculate_reciprocity(nonattackG_10_4)
reciprocity_overall = calculate_reciprocity(overallG_10_4)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.11114537141535691
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.04368825160735852
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 3.84136777841536e-09
Average closeness centrality for attack graph: 0.008337645908803398
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0003025313479083954
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.1016042780748663
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.648181818181818
Average degree centrality for attack graph: 0.016675291817606804
Average degree centrality for non-attack graph: 0.0
Average degree cen